In [6]:
import os
os.environ["JAVA_HOME"] = r"C:\Program Files\Java\jdk-17"
os.environ["PATH"] = r"C:\Program Files\Java\jdk-17\bin;" + os.environ["PATH"]

In [7]:
!java -version

java version "17.0.11" 2024-04-16 LTS
Java(TM) SE Runtime Environment (build 17.0.11+7-LTS-207)
Java HotSpot(TM) 64-Bit Server VM (build 17.0.11+7-LTS-207, mixed mode, sharing)


In [9]:
import h2o
h2o.init(port=54323)

Checking whether there is an H2O instance running at http://localhost:54323..... not found.
Attempting to start a local H2O server...
; Java HotSpot(TM) 64-Bit Server VM (build 17.0.11+7-LTS-207, mixed mode, sharing)
  Starting server from C:\Users\Paul\anaconda3\envs\h2o\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\Paul\AppData\Local\Temp\tmpvtimybmz
  JVM stdout: C:\Users\Paul\AppData\Local\Temp\tmpvtimybmz\h2o_Paul_started_from_python.out
  JVM stderr: C:\Users\Paul\AppData\Local\Temp\tmpvtimybmz\h2o_Paul_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Denver
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,2 months and 13 days
H2O_cluster_name:,H2O_from_python_Paul_efubv5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,29.97 Gb
H2O_cluster_total_cores:,0
H2O_cluster_allowed_cores:,0
H2O_cluster_status:,"locked, healthy"


In [10]:
model_path = "../saved_models\GBM_grid_1_AutoML_1_20241110_153230_model_13"
saved_model = h2o.load_model(model_path)

In [11]:
import pandas as pd
df = pd.read_csv('../exploratory_data_analysis/filtered.csv')
df.columns

C:\Users\Paul\AppData\Local\Temp\ipykernel_32700\4165957757.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../exploratory_data_analysis/filtered.csv')


Index(['Unnamed: 0.1', 'Unnamed: 0', 'State', 'State-County', 'Pollutant',
       'Emissions (Tons)', 'Pollutant Type', 'Site Name', 'EIS Facility ID',
       'Facility Type', 'Street Address', 'NAICS', 'Lat/Lon', 'EPA Region',
       'FIPS', 'StateAbbr_x', 'County_x', 'StateCountyKey', 'Year',
       'StateAbbr_y', 'StateDesc', 'LocationName', 'DataSource', 'Category',
       'Measure', 'Data_Value_Unit', 'Data_Value_Type', 'Data_Value',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'Low_Confidence_Limit', 'High_Confidence_Limit', 'TotalPopulation',
       'TotalPop18plus', 'LocationID', 'CategoryID', 'MeasureId',
       'DataValueTypeID', 'Short_Question_Text', 'Geolocation', 'County_y',
       'Pollutant Type_encoded', 'Pollutant_encoded', 'MeasureId_encoded',
       'CategoryID_encoded', 'TotalPopulation_encoded',
       'TotalPop18plus_encoded', 'State-County_encoded'],
      dtype='object')

In [12]:
input_features = [
    "Pollutant Type_encoded", "Pollutant_encoded", "MeasureId_encoded",
    "CategoryID_encoded", "TotalPopulation_encoded", "TotalPop18plus_encoded",
    "State-County_encoded", "Low_Confidence_Limit", "High_Confidence_Limit", 
    "TotalPopulation", "TotalPop18plus", "Data_Value"
]

output_feature = "Data_Value"

metadata_columns = [
    "State", "State-County", "Pollutant", "Site Name", "EIS Facility ID", 
    "Facility Type", "Street Address", "NAICS", "Lat/Lon", "EPA Region", 
    "FIPS", "StateAbbr_x", "County_x", "StateCountyKey", "Year", 
    "StateAbbr_y", "StateDesc", "LocationName", "DataSource", 
    "Category", "Data_Value_Footnote_Symbol", "Data_Value_Footnote", 
    "Short_Question_Text", "Geolocation", "County_y"
]
h2o_df = h2o.H2OFrame(df)

splits = h2o_df.split_frame(ratios = [0.8], seed = 1)
train = splits[0]
test = splits[1]

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [13]:
predictions = saved_model.predict(test)
print(predictions)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
  predict
  7.1
  7.1
  7.1
  7.10001
  7.10001
  7.10001
  7.1
  7.1
  7.1
  7.10001
[206422 rows x 1 column]



In [14]:
predictions_df = predictions.as_data_frame()  # Convert predictions to Pandas

c:\Users\Paul\anaconda3\envs\h2o\Lib\site-packages\h2o\frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [15]:
test_df = test.as_data_frame()  # Convert predictions to Pandas

c:\Users\Paul\anaconda3\envs\h2o\Lib\site-packages\h2o\frame.py:1981: H2ODependencyWarning: Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using multi-thread, install polars and pyarrow and use it as pandas_df = h2o_df.as_data_frame(use_multi_thread=True)

  warnings.warn("Converting H2O frame to pandas dataframe using single-thread.  For faster conversion using"


In [16]:
combined_df = pd.concat([test_df, predictions_df], axis=1)

In [17]:
combined_df.head()

,Unnamed: 0.1,Unnamed: 0,State,State-County,Pollutant,Emissions (Tons),Pollutant Type,Site Name,EIS Facility ID,Facility Type,...,Geolocation,County_y,Pollutant Type_encoded,Pollutant_encoded,MeasureId_encoded,CategoryID_encoded,TotalPopulation_encoded,TotalPop18plus_encoded,State-County_encoded,predict
0,206,206,Alabama,AL - Autauga,Ammonia,109.72000,CAP,Alabama Power Company,10708711,Electricity Generation via Combustion,...,POINT (-86.6428164158396 32.5350016195151),Autauga,0,43,5,1,2030,2026,28,7.100002
1,766,766,Alabama,AL - Autauga,Carbon Monoxide,4.90000,CAP,Alabama Power Company,10708711,Electricity Generation via Combustion,...,POINT (-86.6428164158396 32.5350016195151),Autauga,0,63,5,1,2030,2026,28,7.100002
2,1646,1646,Alabama,AL - Autauga,Lead Compounds,0.00987,CAP/HAP,Alabama Power Company,10708711,Electricity Generation via Combustion,...,POINT (-86.6428164158396 32.5350016195151),Autauga,1,120,5,1,2030,2026,28,7.100002
3,3006,3006,Alabama,AL - Autauga,Toluene,0.07535,HAP,Autauga County,10583011,Airport,...,POINT (-86.6428164158396 32.5350016195151),Autauga,3,172,5,1,2030,2026,28,7.100006
4,3166,3166,Alabama,AL - Autauga,Acrolein,0.04969,HAP,Autauga County,10583011,Airport,...,POINT (-86.6428164158396 32.5350016195151),Autauga,3,38,5,1,2030,2026,28,7.100006


In [18]:
combined_df.to_csv("../data_model/test_with_predictions.csv", index=False)

In [24]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler()
combined_df['predict_scaled'] = scaler.fit_transform(combined_df[['predict']])
state_avg_df = combined_df.groupby("State").mean('predict').reset_index()
simplified_df = state_avg_df[['State', 'predict_scaled']]
simplified_df.to_csv("../data_model/simplified_test_with_predictions_scaled.csv", index=False)

In [25]:
simplified_df.head()

,State,predict_scaled
0,Alabama,0.595512
1,Alaska,0.509687
2,Arizona,0.702198
3,Arkansas,0.636278
4,California,0.425353


In [26]:
simplified_df = pd.read_csv('../data_model/simplified_test_with_predictions_scaled.csv')

# Convert the DataFrame to JSON
json_data = simplified_df.to_json(orient='records')

# Save the JSON data to a file
with open('../data_model/state_predictions.json', 'w') as file:
    file.write(json_data)

In [ ]:
import json


with open('../visualization/us-states.json') as f:
    states_data = json.load(f)

# Convert predictions to a DataFrame
predictions_df = pd.DataFrame(predictions)

# Extract the properties from the GeoJSON and create a DataFrame
features = states_data['features']
properties = [feature['properties'] for feature in features]
states_df = pd.DataFrame(properties)

# Merge on 'State' and 'name' to add 'predict_scaled' to the states DataFrame
merged_df = states_df.merge(simplified_df, left_on='name', right_on='State', how='left')

merged_df['predict_scaled'] = merged_df['predict_scaled'].fillna(0)

# Update the GeoJSON with predict_scaled values
for feature in states_data['features']:
    state_name = feature['properties']['name']
    predict_scaled = merged_df.loc[merged_df['name'] == state_name, 'predict_scaled'].values
    if len(predict_scaled) > 0:
        feature['properties']['predict_scaled'] = predict_scaled[0]

# Save the updated GeoJSON to a file
with open('updated_statesData.json', 'w') as f:
    json.dump(states_data, f)

: 

In [ ]:
import json

 # Open the file and read its contents
with open('../visualization/us-states.js', 'r') as f:
    js_content = f.read()

# Remove the JavaScript variable declaration and trailing semicolon
json_str = js_content.replace("var statesData =", "").strip().rstrip(';')

# Load the cleaned JSON string
geojson_data = json.loads(json_str)

geojson_df = pd.json_normalize(geojson_data)
geojson_df.head()


,type,features
0,FeatureCollection,"[{'type': 'Feature', 'id': '01', 'properties':..."


In [ ]:
df = pd.read_csv('your_file.csv')

# Convert the DataFrame to JSON
json_data = df.to_json(orient='records')

# Save the JSON data to a file
with open('your_file.json', 'w') as file:
    file.write(json_data)